In [ ]:
# Install required libraries for economic data retrieval and synthetic data generation
!pip install fredapi
!pip install Faker

In [8]:
# Importing essential libraries for data processing, date handling, and logging
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from fredapi import Fred
import os
import logging
from faker import Faker

# Define global constants and configuration values
FRED_API_KEY = '18ff54ec62ffdfc7b64cad1a2a2e7b7c' #update with your fred api key
DATA_PATH = '../Downloads/'  #update with the data path where you want to save the file
CATEGORY = 'Shampoo'
NUM_PRODUCTS = 50
MASTER_FILE = 'US_Historical_Sales.csv'

# Initialize Faker instance for dummy values
fake = Faker()

# Set up logging configuration and ensuring log file exists
LOG_FILE = os.path.join(DATA_PATH, 'data_check_log.txt')
CRITICAL_FIELDS = ['week_start_date', 'product_id', 'units_sold', 'price', 'revenue']

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

if not os.path.isfile(LOG_FILE):
    with open(LOG_FILE, 'w') as f:
        f.write("=== Data Integrity Log Started ===\n")

log_formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(LOG_FILE, mode='a')
file_handler.setFormatter(log_formatter)
logger.addHandler(file_handler)
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)

# Helper to log and print
def log_and_print(message, level='info'):
    print(message)
    getattr(logger, level)(message)
    
# Function to return seasonal adjustment factor based on the month
def seasonal_factor(date):
    if date.month in [6, 7, 8]:
        return 1.2
    elif date.month in [11, 12]:
        return 1.5
    return 1.0

# Function to generate synthetic sales data for all products over the date range
def generate_synthetic_sales(start_date, end_date):
    sundays = pd.date_range(start=start_date, end=end_date, freq='W-SUN')
    products = [{'product_id': f'P{i:03}', 'product_name': f"{CATEGORY} Variant {i}"} for i in range(1, NUM_PRODUCTS + 1)]

    records = []
    for week_start in sundays:
        for product in products:
            base_price = round(np.random.uniform(4.0, 15.0), 2)
            discount = np.random.choice([0, 5, 10, 15, 20], p=[0.5, 0.2, 0.15, 0.1, 0.05])
            price_after_discount = base_price * (1 - discount / 100)
            base_units = np.random.randint(20, 100)
            seasonal_boost = seasonal_factor(week_start)
            discount_boost = 1 + (discount / 100) * 1.5
            units_sold = int(base_units * seasonal_boost * discount_boost)
            revenue = round(units_sold * price_after_discount, 2)

            records.append({
                'week_start_date': week_start,
                'product_id': product['product_id'],
                'product_name': product['product_name'],
                'units_sold': units_sold,
                'price': base_price,
                'discount_percentage': discount,
                'revenue': revenue,
                'region': 'US'
            })

    return pd.DataFrame(records)

# Function to fetch weekly gas prices and CPI data from the FRED API
def fetch_latest_economic_indicators(start_date):
    fred = Fred(api_key=FRED_API_KEY)
    
    # Fetch average weekly gas prices
    gas_prices = fred.get_series('GASREGW')
    gas_df = gas_prices.reset_index()
    gas_df.columns = ['date', 'avg_gas_price']
    gas_df['date'] = pd.to_datetime(gas_df['date'])
    gas_df = gas_df.set_index('date').resample('W-SUN').mean().ffill().reset_index()

    # Fetch Consumer Price Index data
    cpi = fred.get_series('CPIAUCSL')
    cpi_df = cpi.reset_index()
    cpi_df.columns = ['date', 'cpi']
    cpi_df['date'] = pd.to_datetime(cpi_df['date'])
    cpi_df = cpi_df.set_index('date').resample('W-SUN').ffill().reset_index()

    return gas_df, cpi_df

# Function to merges gas prices and CPI data to sales records
def merge_sales_with_economics(df_sales, gas_df, cpi_df):
    df_sales = df_sales.merge(gas_df, left_on="week_start_date", right_on="date", how="left")
    df_sales = df_sales.merge(cpi_df, left_on="week_start_date", right_on="date", how="left")
    df_sales.drop(columns=['date_x', 'date_y'], errors='ignore', inplace=True)
    df_sales['avg_gas_price'] = df_sales['avg_gas_price'].fillna(method='ffill')
    df_sales['cpi'] = df_sales['cpi'].fillna(method='ffill')
    return df_sales

# Main function that controls the workflow: data creation, enrichment, and update
def main():
    today = datetime.today()
    master_path = os.path.join(DATA_PATH, MASTER_FILE)

    # Step 1: Generate master file if it does not already exist
    if not os.path.isfile(master_path):
        logging.info("Master file not found. Generating synthetic data for 2024...")
        start = datetime(2024, 1, 1)
        end = datetime(2024, 12, 31)
        df_master = generate_synthetic_sales(start, end)
        gas_df, cpi_df = fetch_latest_economic_indicators(start)
        df_master = merge_sales_with_economics(df_master, gas_df, cpi_df)
        df_master.to_csv(master_path, index=False)
        logging.info("Master sales file created and saved.")
    else:
        df_master = pd.read_csv(master_path, parse_dates=['week_start_date'])

    # Step 2: Identify next month to generate data (based on existing files)
    existing_months = [f for f in os.listdir(DATA_PATH) if f.startswith("US_Monthly_Sales_") and f.endswith(".csv")]
    existing_dates = []
    for fname in existing_months:
        try:
            dt = datetime.strptime(fname.replace("US_Monthly_Sales_", "").replace(".csv", ""), "%B_%Y")
            existing_dates.append(dt)
        except:
            continue

    last_generated_month = max(existing_dates) if existing_dates else datetime(2024, 12, 1)
    next_month = last_generated_month + pd.offsets.MonthBegin(1)

    # Check if monthly data is already current
    if next_month > today.replace(day=1):
        print(f"Monthly data already up to date. Latest month: {last_generated_month.strftime('%B %Y')}")
        return

    # Step 3: Generate synthetic data for the next month
    start = next_month
    end = (start + pd.offsets.MonthEnd(0)).to_pydatetime()
    df_monthly = generate_synthetic_sales(start, end)
    gas_df, cpi_df = fetch_latest_economic_indicators(start)
    df_monthly = merge_sales_with_economics(df_monthly, gas_df, cpi_df)

    # Save monthly file
    monthly_file = os.path.join(DATA_PATH, f"US_Monthly_Sales_{next_month.strftime('%B_%Y')}.csv")
    df_monthly.to_csv(monthly_file, index=False)
    logging.info(f"Monthly sales file generated and saved: {monthly_file}")

    # Step 4: Update master file by removing existing data for the same month and appending the new data
    month_str = next_month.strftime('%Y-%m')
    df_master = df_master[~df_master['week_start_date'].dt.to_period('M').astype(str).eq(month_str)]
    df_updated = pd.concat([df_master, df_monthly], ignore_index=True)
    df_updated.to_csv(master_path, index=False)
    logging.info(f"Monthly sales for {next_month.strftime('%B %Y')} merged and saved to master database.")
    print(f"Merging complete: Monthly sales for {next_month.strftime('%B %Y')} successfully updated in master data.")

# Main execution block
if __name__ == '__main__':
    main()

Monthly data already up to date. Latest month: June 2025
